In [2]:
!curl ipecho.net/plain
! python -m pip install "pymongo[srv]"==3.11
! pip  install pymongo[srv,tls]
! pip install dnspython
!pip install pandas

34.150.203.201Collecting pymongo[srv]==3.11
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.7/771.7 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 13.2 MB/s eta 0:00:00
  Created wheel for pymongo: filename=pymongo-3.11.0-cp310-cp310-linux_x86_64.whl size=493670 sha256=d767428f065c2770bbff1f3e7b176e9e07f9151f3792103a55049e50db0d02b8
  Stored in directory: /root/.cache/pip/wheels/df/6e/99/27524029bf53fd025e0d44093d77e4fc0b27c6344acd6f3e86
Successfully built pymongo


In [3]:
# @title
from pymongo.mongo_client import MongoClient
from google.colab import userdata

password = userdata.get('mongoDB_password')

uri = "mongodb+srv://PakkyNoi:" + password + "@workshop-bakery.dltqwqx.mongodb.net/?retryWrites=true&w=majority&appName=Workshop-Bakery"
# Create a new client and connect to the server
client = MongoClient(uri)
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)


Pinged your deployment. You successfully connected to MongoDB!


In [20]:
import pandas as pd
from bson import ObjectId

In [11]:
BakeryDB = client.Bakery

CakeCollection = BakeryDB.cakes
CustomerCollection = BakeryDB.customer_details
OrderCollection = BakeryDB.order_records

In [12]:
# get full cake lists

# allCakeList = [data for data in CakeCollection.find()]
allCakeList = list(CakeCollection.find())

allCakeDF = pd.DataFrame(allCakeList)

allCakeDF

,_id,name,shortDescription,description,image,ingredients,recipe,stock
0,64759d513beb40594cb859e9,Chocolate Cake,Chocolate cake is a cake flavored with melted ...,Chocolate cake is made with chocolate; it can ...,https://addapinch.com/wp-content/uploads/2020/...,"[flour, sugar, cocoa powder]",Preheat your oven to the specified temperature...,25
1,64759e4c3beb40594cb859ed,Cheese Cake,Cheesecake is a sweet dessert consisting of on...,Cheesecake is a sweet dessert consisting of on...,https://sallysbakingaddiction.com/wp-content/u...,"[graham cracker crumbs, sugar, eggs, butter, s...",Mix graham cracker crumbs and melted butter fo...,40
2,664758fe06cdc709b70778de,example,example,example,https://url_example.jpeg,"[A, B, C, D]",example,100


In [13]:
allCustomerList = list(CustomerCollection.find())

allCustomerDF = pd.DataFrame(allCustomerList)

allCustomerDF

,_id,firstname,lastname,age,gender,telephone,email,street,villageNumber,city,postal-code
0,664627cdf94f8b7a6dc89b5e,example,example,18,Male,090,example@gmail.com,example,22/2,example,12345
1,664761bd06cdc709b70778df,Veggiee,Pak,18,Male,090,example@gmail.com,example,22/2,example,12345


In [14]:
#get order details
allOrderList = list(OrderCollection.find())

allOrderDF = pd.DataFrame(allOrderList)

# allOrderList
allOrderDF

,_id,firstname,lastname,vegan,menu,quantity,orderingDate,pickupDate,pickupLocation
0,66486239b58df0a4120a2075,Veggiee,Pak,No,Cheese Cake,0,2024-05-18,2024-05-31,Self-pickup
1,66486251b58df0a4120a2076,example,example,No,Chocolate Cake,0,2024-05-18,2024-05-31,Self-pickup


In [29]:
#get customer details from the order

def getCustomerInfoByOrderID(order_id, OrderCollection, CustomerCollection):
    # Find the order document in OrderCollection
    orderDocument = OrderCollection.find_one({'_id': ObjectId(order_id)})

    if not orderDocument:
        return pd.DataFrame()  # Return an empty DataFrame if the order is not found

    # Extract the first name and last name from the order document
    firstname = orderDocument.get('firstname')
    lastname = orderDocument.get('lastname')

    # Find the customer document in CustomerCollection using the first name and last name
    selectedCustomerInfo = list(CustomerCollection.find({
        '$and': [
            {'firstname': firstname},
            {'lastname': lastname}
        ]
    }))

    # Create a DataFrame from the selectedCustomerData
    selectedCustomerInfo = pd.DataFrame(selectedCustomerData)

    return selectedCustomerInfo

In [30]:
wanted_order_id = "66486239b58df0a4120a2075" #@param {type : "string"}

selectedCustomerData = getCustomerInfoByOrderID(wanted_order_id, OrderCollection, CustomerCollection)

selectedCustomerData

,_id,firstname,lastname,age,gender,telephone,email,street,villageNumber,city,postal-code
0,664761bd06cdc709b70778df,Veggiee,Pak,18,Male,090,example@gmail.com,example,22/2,example,12345


In [19]:
# make baking plan

print("baking schedule based on pickup date")
bakingSchedule = pd.DataFrame(list(OrderCollection.aggregate([{"$sort":{"pickupDate": 1}}])))

bakingSchedule

baking schedule based on pickup date


,_id,firstname,lastname,vegan,menu,quantity,orderingDate,pickupDate,pickupLocation
0,66486239b58df0a4120a2075,Veggiee,Pak,No,Cheese Cake,0,2024-05-18,2024-05-31,Self-pickup
1,66486251b58df0a4120a2076,example,example,No,Chocolate Cake,0,2024-05-18,2024-05-31,Self-pickup


In [31]:
# Customer Order History

def getOrderHistoryByCustomerID(order_id, CustomerCollection, OrderCollection):
    # Find the order document in OrderCollection
    customerDocument = CustomerCollection.find_one({'_id': ObjectId(order_id)})

    if not customerDocument:
        return pd.DataFrame()  # Return an empty DataFrame if the order is not found

    # Extract the first name and last name from the order document
    firstname = customerDocument.get('firstname')
    lastname = customerDocument.get('lastname')

    # Find the customer document in CustomerCollection using the first name and last name
    customerOrderHistory = list(OrderCollection.find({
        '$and': [
            {'firstname': firstname},
            {'lastname': lastname}
        ]
    }))

    # Create a DataFrame from the customerOrderHistory
    customerOrderHistory = pd.DataFrame(customerOrderHistory)

    return customerOrderHistory

In [36]:
wanted_customer_id = "664761bd06cdc709b70778df" #@param {type : "string"}

selectedCustomerHistory = getOrderHistoryByCustomerID(wanted_customer_id, CustomerCollection, OrderCollection)

selectedCustomerHistory

,_id,firstname,lastname,vegan,menu,quantity,orderingDate,pickupDate,pickupLocation
0,66486239b58df0a4120a2075,Veggiee,Pak,No,Cheese Cake,0,2024-05-18,2024-05-31,Self-pickup
1,664865c6b58df0a4120a2077,Veggiee,Pak,No,Chocolate Cake,0,2024-05-18,2024-06-08,Delivery
